In [1]:
import numpy as np
import xarray as xr
import crocosi.gridop as gop


%matplotlib inline
from matplotlib import pyplot as plt

from itertools import permutations

In [2]:
#from dask.distributed import Client, LocalCluster
#cluster = LocalCluster()
#client = Client(cluster)
#client

In [3]:
# generate synthetic data
dims = {'time':3, 'z':5, 'y':20, 'x':10}
vmap = {'time':0., 'z':1., 'y':0., 'x':0.}

In [4]:
ndim = 3
def get_P(ndim):
    return [p for p in list(permutations(dims.keys(),ndim)) if 'z' in p]

#for p in P
# ...
get_P(3)

[('time', 'z', 'y'),
 ('time', 'z', 'x'),
 ('time', 'y', 'z'),
 ('time', 'x', 'z'),
 ('z', 'time', 'y'),
 ('z', 'time', 'x'),
 ('z', 'y', 'time'),
 ('z', 'y', 'x'),
 ('z', 'x', 'time'),
 ('z', 'x', 'y'),
 ('y', 'time', 'z'),
 ('y', 'z', 'time'),
 ('y', 'z', 'x'),
 ('y', 'x', 'z'),
 ('x', 'time', 'z'),
 ('x', 'z', 'time'),
 ('x', 'z', 'y'),
 ('x', 'y', 'z')]

In [5]:
def get_ds(p, chunks=None):
    """ Create a synthetic dataset based on some dimension order p
    """

    # assemble coordinates
    _coords = {d: np.arange(dims[d]) for d in p}
    _coords.update(z_target1d=np.arange(.5,6))
    ds = xr.Dataset(coords=_coords)

    # create data veriable and initial grid
    ds['v'] = sum([ds[d]*vmap[d] for d in p])
    ds['z_v'] = ds['z'] + 0.*ds['v']
    # the line above necessarily imposes a dimension order which may vary
    # should loop around all potential dimension order for zv

    # create target grid
    ds['z_target'] = ds['z_target1d']
    # need to vary number of dimensions and their order on the line above
    if chunks:
        ds = ds.chunk(chunks)
    return ds

In [6]:
def test_interp2z_np_3d(ds):
    #print(ds)
    out = gop.interp2z_np_3d(ds['z_target'].values, 
                          (0.*ds['v']+ds['z_v']).values,
                          ds['v'].values, 
                          b_extrap=0, t_extrap=0)
    print('---------')
    print('v shape: {}'.format(list(ds.v.dims)))    
    print('Input shape = {}'.format(ds['v'].values.shape))
    print('Ouput shape = {}'.format(out.shape))
    if out.ndim==3:
        out = out[:,0,0]
    elif out.ndim==2:
        out = out[:,0]
    print(out)
    #hdl = plt.plot(ds['z_target1d'].values, out)
    #plt.grid()
    
test_interp2z_np_3d(get_ds(get_P(3)[6]))
test_interp2z_np_3d(get_ds(get_P(2)[2]))
test_interp2z_np_3d(get_ds(('z',)))

---------
v shape: ['z', 'y', 'time']
Input shape = (5, 20, 3)
Ouput shape = (6, 20, 3)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z', 'y']
Input shape = (5, 20)
Ouput shape = (6, 20)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z']
Input shape = (5,)
Ouput shape = (6,)
[0.5 1.5 2.5 3.5 nan nan]


In [7]:
get_P(4)

[('time', 'z', 'y', 'x'),
 ('time', 'z', 'x', 'y'),
 ('time', 'y', 'z', 'x'),
 ('time', 'y', 'x', 'z'),
 ('time', 'x', 'z', 'y'),
 ('time', 'x', 'y', 'z'),
 ('z', 'time', 'y', 'x'),
 ('z', 'time', 'x', 'y'),
 ('z', 'y', 'time', 'x'),
 ('z', 'y', 'x', 'time'),
 ('z', 'x', 'time', 'y'),
 ('z', 'x', 'y', 'time'),
 ('y', 'time', 'z', 'x'),
 ('y', 'time', 'x', 'z'),
 ('y', 'z', 'time', 'x'),
 ('y', 'z', 'x', 'time'),
 ('y', 'x', 'time', 'z'),
 ('y', 'x', 'z', 'time'),
 ('x', 'time', 'z', 'y'),
 ('x', 'time', 'y', 'z'),
 ('x', 'z', 'time', 'y'),
 ('x', 'z', 'y', 'time'),
 ('x', 'y', 'time', 'z'),
 ('x', 'y', 'z', 'time')]

In [8]:
def test_interp2z_np(ds):
    #print(ds)
    z_pos = ds.v._get_axis_num('z')
    z_size = ds.dims['z']
    out = gop.interp2z_np(ds['z_target'].values, 
                          (0.*ds['v']+ds['z_v']).values,
                          ds['v'].values, 
                          zdim=(z_pos, z_size),
                          b_extrap=0, t_extrap=0)
    print('---------')
    print('v shape: {}'.format(list(ds.v.dims)))
    print('Input shape = {}'.format(ds['v'].values.shape))
    print('Ouput shape = {}'.format(out.shape))
    if out.ndim==4:
        out = out.swapaxes(0,z_pos)[:,0,0,0]
    elif out.ndim==3:
        out = out[:,0,0]
    elif out.ndim==2:
        out = out[:,0]
    print(out)
    
test_interp2z_np(get_ds(get_P(4)[0]))
test_interp2z_np(get_ds(get_P(4)[6]))
test_interp2z_np(get_ds(get_P(3)[6]))
test_interp2z_np(get_ds(get_P(2)[2]))
test_interp2z_np(get_ds(('z',)))

---------
v shape: ['time', 'z', 'y', 'x']
Input shape = (3, 5, 20, 10)
Ouput shape = (3, 6, 20, 10)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z', 'time', 'y', 'x']
Input shape = (5, 3, 20, 10)
Ouput shape = (6, 3, 20, 10)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z', 'y', 'time']
Input shape = (5, 20, 3)
Ouput shape = (6, 20, 3)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z', 'y']
Input shape = (5, 20)
Ouput shape = (6, 20)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z']
Input shape = (5,)
Ouput shape = (6,)
[0.5 1.5 2.5 3.5 nan nan]


In [9]:
def test_interp2z(ds):
    #print(ds)
    out = gop.interp2z(ds['z_target'], ds['v'], 
                          (0.*ds['v']+ds['z_v']),
                          b_extrap=0, t_extrap=0)
    print('---------')
    print('v shape: {}'.format(list(ds.v.dims)))
    print('Input shape = {}'.format(ds['v'].values.shape))
    print('Ouput shape = {}'.format(out.shape))
    if out.ndim==4:
        out = out.swapaxes(0,z_pos)[:,0,0,0]
    elif out.ndim==3:
        out = out[:,0,0]
    elif out.ndim==2:
        out = out[:,0]
    print(out)
    
test_interp2z_np(get_ds(get_P(4)[0]))
test_interp2z_np(get_ds(get_P(4)[6]))
test_interp2z_np(get_ds(get_P(3)[6]))
test_interp2z_np(get_ds(get_P(2)[2]))
test_interp2z_np(get_ds(('z',)))

---------
v shape: ['time', 'z', 'y', 'x']
Input shape = (3, 5, 20, 10)
Ouput shape = (3, 6, 20, 10)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z', 'time', 'y', 'x']
Input shape = (5, 3, 20, 10)
Ouput shape = (6, 3, 20, 10)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z', 'y', 'time']
Input shape = (5, 20, 3)
Ouput shape = (6, 20, 3)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z', 'y']
Input shape = (5, 20)
Ouput shape = (6, 20)
[0.5 1.5 2.5 3.5 nan nan]
---------
v shape: ['z']
Input shape = (5,)
Ouput shape = (6,)
[0.5 1.5 2.5 3.5 nan nan]


In [10]:
def test_interp2z(ds):
    print(ds)
    out = gop.interp2z(ds['z_target'], ds['v'], 
                       (0.*ds['v']+ds['z_v']),
                       zt_dim='z_target1d',
                       b_extrap=0, t_extrap=0)
    print('---------')
    print('v shape: {}'.format(list(ds.v.dims)))
    print('Input shape = {}'.format(ds['v'].values.shape))
    print('Ouput shape = {}'.format(out.shape))
    if out.ndim==4:
        out = out.swapaxes(0,z_pos)[:,0,0,0]
    elif out.ndim==3:
        out = out[:,0,0]
    elif out.ndim==2:
        out = out[:,0]
    print(out)
    
test_interp2z(get_ds(get_P(4)[0], chunks={'x':1}))
test_interp2z(get_ds(get_P(4)[6]))
test_interp2z(get_ds(get_P(3)[6]))
test_interp2z(get_ds(get_P(2)[2]))
test_interp2z(get_ds(('z',)))

<xarray.Dataset>
Dimensions:     (time: 3, x: 10, y: 20, z: 5, z_target1d: 6)
Coordinates:
  * time        (time) int64 0 1 2
  * z           (z) int64 0 1 2 3 4
  * y           (y) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
  * x           (x) int64 0 1 2 3 4 5 6 7 8 9
  * z_target1d  (z_target1d) float64 0.5 1.5 2.5 3.5 4.5 5.5
Data variables:
    v           (time, z, y, x) float64 dask.array<chunksize=(3, 5, 20, 1), meta=np.ndarray>
    z_v         (z, time, y, x) float64 dask.array<chunksize=(5, 3, 20, 1), meta=np.ndarray>
    z_target    (z_target1d) float64 0.5 1.5 2.5 3.5 4.5 5.5


ValueError: Chunks and shape must be of the same length/dimension. Got chunks=(), shape=(2,)

In [5]:
gop.interp2z(ds['z_target'], ds['v'], ds['z_v'])

ValueError: applied function returned data with unexpected number of dimensions: 3 vs 4, for dimensions ('z_target1d', 'time', 'x', 'z')

In [8]:
x = np.array([[1], [2], [3]])
y = np.array([4, 5, 6])
a,b = np.broadcast_arrays(x,y)

In [10]:
x.shape

(3, 1)

In [9]:
y.shape

(3,)

In [9]:
x.shape

(3, 1)

In [8]:
y.shape

(3,)

In [17]:
a.shape==b.shape

True

In [19]:
a.shape[1:]

(3,)